Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence).


Made by [Michelangelo Caretto](https://github.com/rasenqt/computational_intelligence23_24) and [Silvio Chito](https://github.com/SilvioChito/computational_intelligence)    
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [223]:
from random import choices,randint,random, shuffle
import numpy as np
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
import math

# Copyright © 2023 Giovanni Squillero <giovanni.squillero@polito.it>
# https://github.com/squillero/computational-intelligence
# Free for personal or classroom use; see 'LICENSE.md' for details.

from abc import abstractmethod


class AbstractProblem:
    def __init__(self):
        self._calls = 0

    @property
    @abstractmethod
    def x(self):
        pass

    @property
    def calls(self):
        return self._calls

    @staticmethod
    def onemax(genome):
        return sum(bool(g) for g in genome)
     ##Slice the problem  with step sixe of self.x(problme size) and calculate fitness of each slice and then adad together 
     #slice overlaps if self.x>1
     #Fitness Call provide a result based on how similar are the slice based on onemax value
    def __call__(self, genome):
        self._calls += 1
        fitnesses = sorted((AbstractProblem.onemax(genome[s :: self.x]) for s in range(self.x)), reverse=True)
        val = sum(f for f in fitnesses if f == fitnesses[0]) - sum(
            f * (0.1 ** (k + 1)) for k, f in enumerate(f for f in fitnesses if f < fitnesses[0])
        )
        
        return (val / len(genome))
        
    

def make_problem(a):
    class Problem(AbstractProblem):
        @property
        @abstractmethod
        def x(self):
            return a

    return Problem()

### Algorithm parameters

In [224]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 100
LOCI = 1000
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = 0.25
BIT_FLIP_PROBABILITY = 0.15
NUM_GENERATION = 500
OFFSPRING_SIZE_FILTERED=5

In [225]:
class Genome:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float("-inf")
        self.metric = float("-inf") # This metric is used for entropy or hamming distance
       # self.metric = (float, float) #this metric is used for entropy and hamming distance together

In [226]:
def new_offspring(parent1: Genome, parent2: Genome) -> Genome:
    if random() < MUTATION_PROBABILITY:
        return mutate(parent1)
    return three_cut_xover(parent1, parent2)
    
def mutate(parent: Genome) -> Genome:
    new_offspring = deepcopy(parent)

    # Apply bit flip to the shuffled elements
    for _ in range(int(LOCI*0.3)):
        if random() < BIT_FLIP_PROBABILITY:
            index_to_mutate = randint(0,LOCI-1)
            new_offspring.genotype[index_to_mutate] = not new_offspring.genotype[index_to_mutate]
    return new_offspring

def three_cut_xover(ind1: Genome, ind2: Genome) -> Genome:
    one_cut_point = randint(0, int((LOCI)*0.3))
    two_cut_point = randint(int((LOCI)*0.3), int((LOCI)*0.6))
    three_cut_point = randint(int((LOCI)*0.6), LOCI - 1)
  
    # Order the cut points
    cut_points = sorted([one_cut_point, two_cut_point, three_cut_point])
    
    new_ind = Genome()
    new_ind.genotype = (ind1.genotype[:cut_points[0]] +
                        ind2.genotype[cut_points[0]:cut_points[1]] +
                        ind1.genotype[cut_points[1]:cut_points[2]] +
                        ind2.genotype[cut_points[2]:])
    
    assert len(new_ind.genotype) == LOCI
    return new_ind

##HAMMING DISTANCE

def hamming_distance(population: list[Genome]):
    # Calculate Hamming distances
    hamming_distances = np.zeros((len(population), len(population)))

    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            distance = hamming(population[i].genotype, population[j].genotype)
            hamming_distances[i, j] = distance
            hamming_distances[j, i] = distance

    # Calculate mean Hamming distance for each genome
    mean_distances = np.mean(hamming_distances, axis=1)
    return mean_distances 

def hamming_distance_single(individual, population: list[Genome]):
    # Calculate Hamming distances
    hamming_distances = np.zeros(len(population))

    for i in range(len(population)):
        distance = hamming(individual.genotype, population[i].genotype)
        hamming_distances[i] = distance

    # Calculate mean Hamming distance for each genome
    mean_distance = np.mean(hamming_distances)
    return mean_distance

def sort_by_remain(population: list[Genome]):
    tmp_population=deepcopy(population)
   
    for i in tmp_population:
        i.hamming_distance=(sum(i.genotype)%5)  
    tmp_population.sort(key=lambda i: i.hamming_distance, reverse=False)
    return tmp_population

def sort_by_hamming_distance(population):
        tmp_population=deepcopy(population)
        for x in tmp_population:
            x.metric=hamming_distance_single(x, population)
        tmp_population.sort(key=lambda i: i.metric, reverse=True)
        return tmp_population
    

## ENTROPY RULES 
def sort_by_entropy(population: list[Genome]):
    tmp_population=deepcopy(population)

    for individual in tmp_population:
        individual.metric = calculate_entropy(individual.genotype)

    tmp_population.sort(key=lambda i: i.metric, reverse=False)
    return tmp_population

def calculate_entropy(vector):
    total_elements = len(vector)
    # Calcola la frequenza di ciascun valore nel vettore
    counts = {0: 0, 1: 0}
    for element in vector:
        counts[element] += 1

    # Calcola le probabilità
    probabilities = {key: count / total_elements for key, count in counts.items()}

    # Calcola l'entropia
    entropy = -sum(p * math.log2(p) if p != 0 else 0 for p in probabilities.values())
    return "{:.4f}".format(entropy)

def evaluate_population(population, already_computed):
    tmp_population=deepcopy(population)
    for p in tmp_population:
        key_already_computed = tuple(p.genotype)
        if key_already_computed in already_computed:
            p.fitness = already_computed[key_already_computed]
        else:
            p.fitness = fitness(p.genotype)
            already_computed[key_already_computed] = p.fitness
    return tmp_population

def evaluate_population_nocache(population):
    tmp_population=deepcopy(population)
    for p in tmp_population:
        p.fitness = fitness(p.genotype)
    return tmp_population

def sort_by_entropy_and_hamming(population: list[Genome]):
    tmp_population=deepcopy(population)

    for individual in tmp_population:
        my_list = list(individual.metric)
        my_list[1] = -calculate_entropy(individual.genotype)
        my_list[0] = hamming_distance_single(individual, population)
        my_tuple = tuple(my_list)
        individual.metric = my_tuple

    tmp_population.sort(key=lambda i: i.metric, reverse=True)
    return tmp_population

### Vanilla EA (NO entropy, NO solution caching)

In [227]:
Problem_Istance=[1,2,5,10]
for value in Problem_Istance:
        fitness = make_problem(value)
        population = [Genome() for _ in range(POPULATION_SIZE)] # starting pouplation of POPULATION_SIZE individuals
        population=evaluate_population_nocache(population)
        population.sort(key=lambda i: i.fitness, reverse=True)
        best_fitness = population[0].fitness
        gen = 0

        while  gen < NUM_GENERATION:                
                for _ in range(OFFSPRING_SIZE):
                        offspring = new_offspring(population[0],population[1])
                        offspring.fitness=fitness(offspring.genotype)
                population.extend([offspring])
                population.sort(key=lambda i: i.fitness, reverse=True)
                population = population[:POPULATION_SIZE]
                best_fitness=population[0]
                gen += 1
        print(f"ProblemIstance->{value}")
        print(f"Fitness Value->{population[0].fitness:.2%}")
        print(f"Fitness Calls->{fitness.calls}")
        print("-----------------------------")
        

ProblemIstance->1
Fitness Value->60.30%
Fitness Calls->50050
-----------------------------
ProblemIstance->2
Fitness Value->51.60%
Fitness Calls->50050
-----------------------------
ProblemIstance->5
Fitness Value->21.88%
Fitness Calls->50050
-----------------------------
ProblemIstance->10
Fitness Value->12.99%
Fitness Calls->50050
-----------------------------


### EA (WITH entropy AND solution caching)

In [228]:
Problem_Istance=[1,2,5,10]
for value in Problem_Istance:
        fitness = make_problem(value)         
        already_computed = {}

        population = [Genome() for _ in range(POPULATION_SIZE)] # starting pouplation of POPULATION_SIZE individuals
        population = evaluate_population(population, already_computed) # computes fitness for every individual

        population.sort(key=lambda i: i.fitness, reverse=True)
        best_fitness = population[0].fitness
        gen = 0

        while gen < NUM_GENERATION:
                offspring=[]
                ##create child
                for i in range(OFFSPRING_SIZE):
                        child = new_offspring(population[0],population[1])
                        offspring.append(child)
                #evaluate child by entropy metric
                offspring = sort_by_entropy(offspring)
                #offspring = sort_by_hamming_distance(offspring)
                #offspring = sort_by_entropy_and_hamming(offspring)
                offspring = offspring[:OFFSPRING_SIZE_FILTERED]
                offspring = evaluate_population(offspring, already_computed)
                population.extend(offspring)
                population.sort(key=lambda i: i.fitness, reverse=True)
                population=population[:POPULATION_SIZE]
        
                best_fitness = population[0]
                #print(best_fitness.genotype)
                gen += 1

        print(f"ProblemIstance->{value}")
        print(f"Fitness Value->{population[0].fitness:.2%}")
        print(f"Fitness Calls->{fitness.calls}")
        print("-----------------------------")

ProblemIstance->1
Fitness Value->74.30%
Fitness Calls->236
-----------------------------
ProblemIstance->2
Fitness Value->67.80%
Fitness Calls->654
-----------------------------
ProblemIstance->5
Fitness Value->27.60%
Fitness Calls->889
-----------------------------
ProblemIstance->10
Fitness Value->28.90%
Fitness Calls->1686
-----------------------------
